In [1]:
import os
import tiktoken
import math
from tokenizers import Tokenizer
from PyPDF2 import PdfReader
import tqdm as notebook_tqdm

In [2]:
# Define the path to the "docs" folder (adjust if needed)
docs_folder = "docs"

# Get all files in the folder
files = os.listdir(docs_folder)

# Look for a PDF file
for filename in files:
  if filename.endswith(".pdf"):
    # Construct the full path to the PDF
    pdf_path = os.path.join(docs_folder, filename)

    # Open the PDF with PyPDF2
    with open(pdf_path, 'rb') as pdf_file:
      pdf_reader = PdfReader(pdf_file)

      # Extract all text content
      text = ""
      for page in pdf_reader.pages:
        text += page.extract_text()

      # Store text content in the doc variable
      doc = text
      break  # Exit the loop after finding the first PDF

# Handle the case where no PDF is found
else:
  print("No PDF file found in the 'docs' folder.")

# Now you can use the 'doc' variable containing the extracted text
print(f"Extracted text: {doc[:100]}...")  # Print first 100 characters

Extracted text: Attention Is All You Need
Ashish Vaswani
Google Brain
avaswani@google.comNoam Shazeer
Google Brain...


In [11]:
# load model and tokenizer
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
model_name = "bert-base-uncased"
#model_name = "cl100k_base"

#encoding = tiktoken.get_encoding("cl100k_base")
#print(encoding)
print(tokenizer)

In [12]:
def auto_chunker(document, max_chunk_size, model):

    tokenizer = Tokenizer.from_pretrained(model)
    #tokenizer = tiktoken.encoding_for_model(model)
    document_tokens = tokenizer.encode(document)
    document_size = len(document_tokens)
    # total chunk number
    K = math.ceil(document_size / max_chunk_size)
    # average integer chunk size
    average_chunk_size = math.ceil(document_size / K)
    # number of chunks with average_chunk_size - 1 
    shorter_chunk_number = K * average_chunk_size - document_size
    # number of chunks with average_chunk_size
    standard_chunk_number = K - shorter_chunk_number

    chunks = []
    chunk_start = 0
    for i in range(0, K):
        if i < standard_chunk_number:
            chunk_end = chunk_start + average_chunk_size
        else:
            chunk_end = chunk_start + average_chunk_size - 1
        chunk = document_tokens[chunk_start:chunk_end]
        chunks.append(tokenizer.decode(chunk))
        chunk_start = chunk_end

    assert chunk_start == document_size
    return chunks

In [13]:
chunks = auto_chunker(doc, 1024, tokenizer)
print(chunks)

TypeError: argument 'identifier': 'Tokenizer' object cannot be converted to 'PyString'